In [1]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from sklearn.linear_model import LinearRegression as LR
# from tqdm import tqdm
# from tensorflow.keras.utils import Sequence
# from sklearn.model_selection import train_test_split
# import math

# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, Dropout
# from tensorflow.keras.metrics import RootMeanSquaredError
# from tensorflow.keras.models import Model
# from tensorflow.keras.regularizers import L1, L2
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

# class XGB():
#     def __init__(self, params, columns):
#         self.params = params
#         self.columns = columns
#         pass
    
#     def cross_val_predict(self, df, skn, n_folds=5):
#         assert 'inner_fold' in df.columns, 'define fold with column name "inner_fold"'
#         df_station = df[df['skn'] == skn]
        
#         list_ytrue = []
#         list_ypred = []
#         for k in range(n_folds):
#             df_train = df_station[df_station['inner_fold'] != k]
#             df_test = df_station[df_station['inner_fold'] == k]
#             x_train, x_test = np.array(df_train[self.columns]), np.array(df_test[self.columns])
#             y_train, y_test = np.array(df_train['data_in']), np.array(df_test['data_in'])
            
#             model = XGBRegressor(**self.params)
#             model.fit(x_train, y_train)
            
#             y_pred = model.predict(x_test)
            
#             list_ytrue.extend(y_test)
#             list_ypred.extend(y_pred)
#         return {
#             "mae": mean_absolute_error(list_ytrue, list_ypred),
#             "rmse": mean_squared_error(list_ytrue, list_ypred, squared=False)
#         }
    
#     def evaluate_by_station(self, df_train, df_test, skn, n_iter=1):
#         rmse = []
#         for _ in range(n_iter):
#             df_train_station = df_train[df_train['skn'] == skn]
#             df_test_station = df_test[df_test['skn'] == skn]
            
#             x_train, x_test = np.array(df_train_station[self.columns]), np.array(df_test_station[self.columns])
#             y_train, y_test = np.array(df_train_station['data_in']), np.array(df_test_station['data_in'])
#             # print("hsere", self.params)
#             params = self.params
#             params['seed'] = np.random.randint(100)
#             # params['objective'] = 'reg:squarederror'
#             model = XGBRegressor(params)
#             model.fit(x_train, y_train)
#             y_pred = model.predict(x_test)
#             rmse.append(mean_squared_error(y_test, y_pred, squared=False))
#         print(rmse)
#         return {
#                 "skn": skn,
#                 "n_iter": n_iter,
#                 "rmse_xgb": np.mean(rmse),
#                 "rmse_std_xgb": np.std(rmse)
#         }
    
#     def evaluate(self, df_train, df_test, n_iter=1):
#         ret_vals = []
#         for skn in df_train['skn'].unique():
#             r = self.evaluate_by_station(df_train, df_test, skn, n_iter)
#             ret_vals.append(r)
#         return pd.DataFrame(ret_vals)
# #             df_train_station = df_train[df_train['skn'] == skn]
# #             df_test_station = df_test[df_test['skn'] == skn]

# #             x_train, x_test = np.array(df_train_station[self.columns]), np.array(df_test_station[self.columns])
# #             y_train, y_test = np.array(df_train_station['data_in']), np.array(df_test_station['data_in'])

# #             model = XGBRegressor(**self.params)
# #             model.fit(x_train, y_train)
# #             y_pred = model.predict(x_test)
# #             ret_vals.append(
# #                 {
# #                     "skn": skn,
# #                     "rmse_xgb": mean_squared_error(y_test, y_pred, squared=False),
# #                     "mae_xgb": mean_absolute_error(y_test, y_pred)
# #                 }
# #             )
# #         return pd.DataFrame(ret_vals)

2022-08-19 22:33:04.363070: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import pandas as pd
import sys
sys.path.append('/home/yusukemh/github/yusukemh/StatisticalDownscaling/writeup')
from hyperparameters import XGB_PARAMS
from util import load_data, XGB
from config import C_COMMON, C_GRID, C_SINGLE, FILENAME


2022-08-19 22:35:14.194259: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
columns = C_SINGLE
for p in [200]:
    for item in XGB_PARAMS:
        skn = item['skn']
        # print(item['skn'], item['params'])
        df_train, df_test = load_data(columns + C_COMMON, FILENAME)

        df_train = df_train[df_train['skn'] == skn]
        df_test = df_test[df_test['skn'] == skn]

        df_train = df_train.iloc[-p:]
        # test:
        item['params']['subsample'] = 0.5
        # print(item['params'])
        station_model = XGB(
            columns=columns,
            params=item['params'],
        )
        r = station_model.evaluate_by_station(df_train, df_test, skn=skn, n_iter=5)

[5.161916833397358, 5.224048107000542, 5.194233011290804, 5.277197406219743, 5.252201952767221]
[7.794358775864844, 7.642412347241461, 7.61274365486013, 7.45605023266328, 7.7967487288596935]
[5.735594622710229, 5.728765525996752, 5.855023891749425, 5.716765171988648, 5.807755610739093]
[2.1881998158865277, 2.1861165187181633, 2.200306178172383, 2.1812968798896115, 2.2003989423017614]
[2.082550398778159, 2.0488788570259495, 2.05453905736435, 2.097949340484944, 2.082550398778159]
[1.638387283667129, 1.617272927979073, 1.5737816530527222, 1.68193969109388, 1.5291239662439895]
[1.1620110989455643, 1.1516071169558353, 1.1520799573371103, 1.1533754000820646, 1.1391749857042106]
[2.708578342214036, 2.8611651429879297, 2.7015504873665006, 2.8281787215787597, 2.807288891827532]
[1.364862360716869, 1.4136270197343843, 1.379063505551719, 1.38765042994042, 1.3865197401102847]
[1.4470012857987982, 1.4870482029776801, 1.45626753243716, 1.4995495851845484, 1.4918834190728862]
[1.746598322887454, 1.79